<a href="https://colab.research.google.com/github/Mehulsoni26/Uncertainty_Quantification_LLMs/blob/main/Uncertainty_Quantification_LLAMA2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

import os
dir_path = '/content/drive/MyDrive/Uncertainty_Quantification'
os.chdir(dir_path)

In [ ]:
%ls

'Copy of Uncertainty_Quantification_llama.ipynb'   Uncertainty_Quantification_1.ipynb
 data_normal.csv                                  'Uncertainty_Quantification_llama (1).ipynb'
 data_with_stress_index.csv                        Uncertainty_Quantification_llama.ipynb
 Sample_data_creation_stress_index.ipynb


In [ ]:
%%capture
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install evaluate
!pip install -qqq trl==0.7.1
!pip install torch

In [ ]:
import torch
import time
import evaluate
import pandas as pd
import numpy as np
from datasets import Dataset, load_dataset
import random
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [ ]:
import getpass

# Prompt for the Hugging Face token
hf_token = getpass.getpass("Enter your Hugging Face token: ")

import os
os.environ['HUGGINGFACE_TOKEN'] = hf_token

Enter your Hugging Face token: ··········


In [ ]:
# model_id =  "NousResearch/Llama-2-7b-hf"
model_id = "meta-llama/Llama-2-7b-chat-hf"
# model_id = "mistralai/Mistral-7B-v0.1"
# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16
# )
quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map="auto",token=hf_token)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id,token=hf_token)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
# from transformers import StoppingCriteria
# class EosListStoppingCriteria(StoppingCriteria):
#     def __init__(self, eos_sequence = [319,350,360,315]):
#         self.eos_sequence = eos_sequence

#     def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
#         last_ids = input_ids[:,-len(self.eos_sequence):].tolist()
#         return self.eos_sequence in last_ids

In [ ]:
import warnings
warnings.filterwarnings('ignore')

prompt = f"""
###Instruction: Act as a science expert and help in answering the following multiple choice question. The answer should be one of the option choices.
Follow the pattern from the first example and answer the second question accordingly.

###Question:
This is a question from high school biology.A piece of potato is dropped into a beaker of pure water.
Which of the following describes the activity after the potato is immersed into the water?
(A) Water moves from the potato into the surrounding water.
(B) Water moves from the surrounding water into the potato.
(C) Potato cells plasmolyze.
(D) Solutes in the water move into the potato.

###Answer:
B

###Question:
You are the world’s best expert in high school biology. From the solubility rules, which of the following is true?
(A) All hydroxides are soluble
(B) All sulfates are soluble
(C) All chlorides, bromides, and iodides are soluble
(D) All ammonium-containing compounds are soluble

###Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
# output = tokenizer.decode(
#     model.generate(
#         inputs["input_ids"],
#         temperature=0.2,
#         do_sample=True,
#         max_new_tokens=2,
#     )[0],
#     skip_special_tokens=False
# )

generated_ids=model.generate(
        inputs["input_ids"],
        temperature=0.2,
        do_sample=True,
        max_new_tokens=2)

# dash_line = '-'.join('' for x in range(100))
# print(dash_line)
# print(f'MODEL GENERATION - ONE SHOT:\n{output}')

In [ ]:
generated_ids.shape

torch.Size([1, 273])

In [ ]:
prompt = f"""
###Instruction: Act as a science expert and help in answering the following multiple choice question. The answer should be one of the option choices.
Follow the pattern from the first example and answer the second question accordingly.

###Question:
This is a question from high school biology.A piece of potato is dropped into a beaker of pure water.
Which of the following describes the activity after the potato is immersed into the water?
(A) Water moves from the potato into the surrounding water.
(B) Water moves from the surrounding water into the potato.
(C) Potato cells plasmolyze.
(D) Solutes in the water move into the potato.

###Answer:
B

###Question:
You are the world’s best expert in high school biology. From the solubility rules, which of the following is true?
(A) All hydroxides are soluble
(B) All sulfates are soluble
(C) All chlorides, bromides, and iodides are soluble
(D) All ammonium-containing compounds are soluble

###Answer:
"""

inputs = tokenizer(prompt, return_tensors='pt')
outputs=model(inputs["input_ids"])



/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


In [ ]:
next_token_logits = outputs.logits[:, -1]

In [ ]:
next_token_logits

tensor([[-10.9375, -11.1562,   5.4258,  ...,  -6.5586,  -5.6289,  -3.4941]],
       grad_fn=<SelectBackward0>)

In [ ]:
def softmax(x):
  e_x=np.exp(x-np.max(x))
  return e_x/e_x.sum()


tokens_of_interest=['A','B','C','D']
token_indices = tokenizer.convert_tokens_to_ids(tokens_of_interest)
indices_in_logits = {token: next_token_logits[0,token_idx].item() for token, token_idx in zip(tokens_of_interest, token_indices)}

label_to_softmax_dict=dict(zip(tokens_of_interest,softmax(np.array(list(indices_in_logits.values())))))

print(label_to_softmax_dict)

In [ ]:
indices_in_logits

{'A': 21.578125, 'B': 20.78125, 'C': 21.8125, 'D': 21.5}

In [ ]:
label_to_softmax_dict

{'A': 0.274747724454225,
 'B': 0.12383850168956001,
 'C': 0.347313666960872,
 'D': 0.25410010689534285}

In [ ]:
df_stress_index=pd.read_csv('data_with_stress_index.csv',usecols=lambda col:col not in ['Unnamed: 0'] )

In [ ]:
# df_stress_index

Bucketing the stress index into 10 different segments (option choices)

In [ ]:
[str({i:j for i,j in row.to_dict().items() if i not in ['stress_index','stress_index_bucket']}) for _, row in df_stress_index.head(2).iterrows()]

["{'news_sentiment': 0.3931444486731639, 'broker_count_imbalance': 0.1243619871996783, 'volume_indicator': 0.6778892697073087, 'benchmark_price_difference': 0.4977762417709135, 'trade_count_imbalance': -0.7206982745760815, 'one_sided_trade_indicator': -0.6800763028141159, 'tranche_size_indicator': 0.7248691201811168}",
 "{'news_sentiment': 0.7724956104208642, 'broker_count_imbalance': 0.3693763241525458, 'volume_indicator': -0.8772379830243947, 'benchmark_price_difference': 0.1397481154484952, 'trade_count_imbalance': 0.8518638934191982, 'one_sided_trade_indicator': -0.2575187071551993, 'tranche_size_indicator': 0.5860627411447076}"]

In [ ]:
df_stress_index.loc[:,'stress_index_bucket']=pd.cut(df_stress_index['stress_index'],bins=5,labels=[str(np.round(x,1))+'-'+str(np.round(x+0.2,1)) for x in np.arange(0,1,0.2)])

In [ ]:
df_stress_index.head()

,news_sentiment,broker_count_imbalance,volume_indicator,benchmark_price_difference,trade_count_imbalance,one_sided_trade_indicator,tranche_size_indicator,stress_index,stress_index_bucket
0,0.393144,0.124362,0.677889,0.497776,-0.720698,-0.680076,0.724869,0.551515,0.4-0.6
1,0.772496,0.369376,-0.877238,0.139748,0.851864,-0.257519,0.586063,0.623805,0.6-0.8
2,-0.189353,0.190013,-0.485352,-0.364914,-0.183833,-0.235899,0.356616,0.305677,0.2-0.4
3,0.219090,-0.577930,0.876170,0.495462,0.182495,0.094340,0.106455,0.599768,0.4-0.6
4,0.038766,0.474529,0.345526,-0.153370,-0.020692,0.238055,0.432594,0.594587,0.4-0.6


**Trading Bot Prompt**

In [ ]:
import random
random.seed(42)

In [ ]:
def shuffle_list(original_list):
    # Create a copy of the list
    shuffled_list = original_list.copy()

    # Shuffle the copy
    random.shuffle(shuffled_list)

    return shuffled_list

In [ ]:
list_stress_index_bucket=[str(np.round(x,1))+'-'+str(np.round(x+0.2,1)) for x in np.arange(0,1,0.2)]
list_option_choices=[chr(x) for x in range(65,65+len(list_stress_index_bucket))]

In [ ]:
# Note : Act as a finance expert and help in answering the following multiple choice question. The answer should be one of the option choices.
# Follow the pattern from the first example and answer the second question accordingly.

In [ ]:
# instruction="""
# ###Instruction:
# You are fixed income corporate bond trader. Your need to use the feature values that I will provide to you and select the correct option for classifying the stress index (with a range of 0 to 1) into one of the defined buckets.
# Feature descriptions:
# - news_sentiment: Scale from -1 (negative) to +1 (positive) reflecting the sentiment in news about the bond issuer over the past 7 days.
# - broker_count_imbalance: Difference in broker count buying or selling the same security in the last 2 days, ranging from -1 (more selling) to +1 (more buying).
# - volume_indicator: Indicates if a bond is heavily traded in the last 2 days, with -1 for high selling volume and +1 for high buying volume.
# - benchmark_price_difference: Compares a bond's quoted price to the benchmark, ranging from -1 (below benchmark) to +1 (above benchmark), indicating price stress.
# - trade_count_imbalance: Difference in buy and sell trades of a security over 2 days, from -1 (more sells) to +1 (more buys).
# - one_sided_trade_indicator: Imbalance in buy or sell trades over 7 days, with -1 for predominantly sell trades and +1 for buy trades.
# - tranche_size_indicator: Assesses bond stress by the direction of trades and tranche size, ranging from 0 (large tranche) to 1 (small tranche).
# ###Question:
# Given the feature value dict as:
# {'news_sentiment': 0.3931444486731639, 'broker_count_imbalance': 0.1243619871996783, 'volume_indicator': 0.6778892697073087, 'benchmark_price_difference': 0.4977762417709135, 'trade_count_imbalance': -0.7206982745760815, 'one_sided_trade_indicator': -0.6800763028141159, 'tranche_size_indicator': 0.7248691201811168}
# Return the correct option for classifying stress index from the following options based on above features. Do not generate anything else apart from the option.
# """

In [ ]:
instruction="""
###Instruction:
You are fixed income corporate bond trader. Your need to use the features that I will provide and select the correct option for stress index bucket (with a range of 0 to 1).
###Question:
Given the feature value dict as:
{'news_sentiment': 0.3931444486731639, 'broker_count_imbalance': 0.1243619871996783, 'volume_indicator': 0.6778892697073087, 'benchmark_price_difference': 0.4977762417709135, 'trade_count_imbalance': -0.7206982745760815, 'one_sided_trade_indicator': -0.6800763028141159, 'tranche_size_indicator': 0.7248691201811168}
Return the correct option for stress index from the options based on above features. Do not generate anything else apart from the option.
"""

In [ ]:
choice_prompt=''''''
for i,j in zip(list_option_choices,shuffle_list(list_stress_index_bucket)):
  choice_prompt+=f'({i}) {j}\n'

In [ ]:
example_prompt=''''''
example_prompt+=instruction
example_prompt+=choice_prompt
example_prompt+='''
###Answer:
B
'''

In [ ]:
print(example_prompt)


###Instruction:
You are fixed income corporate bond trader. Your need to use the features that I will provide and select the correct option for stress index bucket (with a range of 0 to 1).
###Question:
Given the feature value dict as:
{'news_sentiment': 0.3931444486731639, 'broker_count_imbalance': 0.1243619871996783, 'volume_indicator': 0.6778892697073087, 'benchmark_price_difference': 0.4977762417709135, 'trade_count_imbalance': -0.7206982745760815, 'one_sided_trade_indicator': -0.6800763028141159, 'tranche_size_indicator': 0.7248691201811168}
Return the correct option for stress index from the options based on above features. Do not generate anything else apart from the option.
(A) 0.4-0.6
(B) 0.2-0.4
(C) 0.0-0.2
(D) 0.6-0.8
(E) 0.8-1.0

###Answer:
B



In [ ]:
final_prompt=''''''
final_prompt+=example_prompt
final_prompt+='''
###Question:
Given the feature value dict as:
{'news_sentiment': 0.7724956104208642, 'broker_count_imbalance': 0.3693763241525458, 'volume_indicator': -0.8772379830243947, 'benchmark_price_difference': 0.1397481154484952, 'trade_count_imbalance': 0.8518638934191982, 'one_sided_trade_indicator': -0.2575187071551993, 'tranche_size_indicator': 0.5860627411447076}
Return the correct option for classifying stress index from the following options based on above features. Do not generate anything else apart from the option.
'''
choice_prompt=''''''
for i,j in zip(list_option_choices,shuffle_list(list_stress_index_bucket)):
  choice_prompt+=f'({i}) {j}\n'

final_prompt+=choice_prompt
final_prompt+='''
###Answer:
'''

In [ ]:
print(final_prompt)


###Instruction:
You are fixed income corporate bond trader. Your need to use the features that I will provide and select the correct option for stress index bucket (with a range of 0 to 1).
###Question:
Given the feature value dict as:
{'news_sentiment': 0.3931444486731639, 'broker_count_imbalance': 0.1243619871996783, 'volume_indicator': 0.6778892697073087, 'benchmark_price_difference': 0.4977762417709135, 'trade_count_imbalance': -0.7206982745760815, 'one_sided_trade_indicator': -0.6800763028141159, 'tranche_size_indicator': 0.7248691201811168}
Return the correct option for stress index from the options based on above features. Do not generate anything else apart from the option.
(A) 0.4-0.6
(B) 0.2-0.4
(C) 0.0-0.2
(D) 0.6-0.8
(E) 0.8-1.0

###Answer:
B

###Question:
Given the feature value dict as:
{'news_sentiment': 0.7724956104208642, 'broker_count_imbalance': 0.3693763241525458, 'volume_indicator': -0.8772379830243947, 'benchmark_price_difference': 0.1397481154484952, 'trade_coun

**Next Token Logits**

In [ ]:
inputs = tokenizer(final_prompt, return_tensors='pt')
generated_ids=model.generate(
        inputs["input_ids"].to('cuda'),
        temperature=0.2,
        do_sample=True,
        max_new_tokens=1)

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 113.06 MiB is free. Process 2074 has 14.63 GiB memory in use. Of the allocated memory 13.90 GiB is allocated by PyTorch, and 626.19 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
generated_ids.shape

torch.Size([1, 691])

In [ ]:
output = tokenizer.decode(
    generated_ids[0],
        # stopping_criteria = [EosListStoppingCriteria()] ,
    skip_special_tokens=False
)

In [ ]:
print(output)

<s> 
###Instruction:
You are fixed income corporate bond trader. Your need to use the feature values that I will provide to you and select the correct option for classifying the stress index (with a range of 0 to 1) into one of the defined buckets.
###Question:
Given the feature value dict as:
{'news_sentiment': 0.3931444486731639, 'broker_count_imbalance': 0.1243619871996783, 'volume_indicator': 0.6778892697073087, 'benchmark_price_difference': 0.4977762417709135, 'trade_count_imbalance': -0.7206982745760815, 'one_sided_trade_indicator': -0.6800763028141159, 'tranche_size_indicator': 0.7248691201811168}
Return the correct option for classifying stress index from the following options based on above features. Do not generate anything else apart from the option.
(A) 0.2-0.4
(B) 0.4-0.6
(C) 0.6-0.8
(D) 0.8-1.0
(E) 0.0-0.2

###Answer:
B

###Question:
Given the feature value dict as:
{'news_sentiment': 0.7724956104208642, 'broker_count_imbalance': 0.3693763241525458, 'volume_indicator': -0

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
outputs=model(inputs["input_ids"])
# next_token_logits = outputs.logits[:, -1]

OutOfMemoryError: CUDA out of memory. Tried to allocate 172.00 MiB. GPU 0 has a total capacty of 14.75 GiB of which 113.06 MiB is free. Process 2074 has 14.63 GiB memory in use. Of the allocated memory 13.83 GiB is allocated by PyTorch, and 695.46 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
def softmax(x):
  e_x=np.exp(x-np.max(x))
  return e_x/e_x.sum()


token_indices = tokenizer.convert_tokens_to_ids(list_option_choices)
indices_in_logits = {token: next_token_logits[0,token_idx].item() for token, token_idx in zip(list_option_choices, token_indices)}

label_to_softmax_dict=dict(zip(tokens_of_interest,softmax(np.array(list(indices_in_logits.values())))))

print(label_to_softmax_dict)